In [1]:
import pdfplumber
import pandas as pd
import matplotlib.pyplot as plt
import re
import plotly.express as px
import plotly.graph_objects as go
import numpy as np
import sys
import os
sys.path.append(os.path.abspath('..'))

plt.style.use('default')

In [4]:
revenue_path = 'z_Data/ME_Revenue/FY 2025 Revenue ME.pdf'

In [5]:
# Extract table from page 6 as text
with pdfplumber.open(revenue_path) as pdf:
    exhibit_i_text = pdf.pages[5].extract_text()  # Page 6 is index 5

# Find table boundaries
lines = exhibit_i_text.split('\n')
start_idx = None
end_idx = None
for i, line in enumerate(lines):
    if line.startswith('Sales and Use Tax'):
        start_idx = i
    if line.startswith('NOTES:'):
        end_idx = i
        break

# Extract table lines
table_lines = [line.strip() for line in lines[start_idx:end_idx] if line.strip()]

# Parse table data - each row has 10 elements: source + 9 values
data = []
for i in range(0, len(table_lines), 10):
    if i + 9 < len(table_lines):
        row_lines = table_lines[i:i+10]
        source = row_lines[0]
        values = row_lines[1:]
        
        # Clean values: remove $, %, commas, handle negatives
        cleaned_values = []
        for val in values:
            val = val.replace('$', '').replace('%', '').replace(',', '')
            if val.startswith('(') and val.endswith(')'):
                val = '-' + val[1:-1]
            try:
                if '.' in val:
                    cleaned_values.append(float(val))
                else:
                    cleaned_values.append(int(val))
            except ValueError:
                cleaned_values.append(0)  # Default to 0 if not parseable
        
        row = [source] + cleaned_values
        data.append(row)

# Create DataFrame
columns = ['Source', 'Month Actual', 'Month Budget', 'Month Variance', 'Month %', 'FYTD Actual', 'FYTD Budget', 'FYTD Variance', 'FYTD %', 'Total Budgeted FY']
df_text = pd.DataFrame(data, columns=columns)
df_text

FileNotFoundError: [Errno 2] No such file or directory: 'z_Data/ME_Revenue/FY 2025 Revenue ME.pdf'

In [6]:
def find_exhibit_page(pdf, exhibit_name):
    for i, page in enumerate(pdf.pages):
        lines = page.extract_text_lines()
        first_line = lines[0]['text'] if lines else ""
        if first_line.endswith(exhibit_name) or first_line.endswith(exhibit_name.upper()):
            return i
    return -1

In [ ]:
# Extract table from page 6 as text
def extract_revenue_source_table(year):
    revenue_path = f'../z_Data/ME_Revenue/FY {year} Revenue ME.pdf'
    with pdfplumber.open(revenue_path) as pdf:
        exhibit_i_page = find_exhibit_page(pdf, 'Exhibit I')
        if( exhibit_i_page == -1):
            raise ValueError("Exhibit I page not found.")
        exhibit_i_text = pdf.pages[exhibit_i_page].extract_text()  # Page 6 is index 5

    # Find table boundaries
    lines = exhibit_i_text.split('\n')
    start_idx = None
    end_idx = None
    for i, line in enumerate(lines):
        if line.startswith('Sales and Use Tax'):
            start_idx = i
        if line.startswith('NOTES:'):
            end_idx = i
            break

    # Extract table lines
    table_lines = [line.strip() for line in lines[start_idx:end_idx] if line.strip()]

    data = []
    for line in table_lines:
        line = line.replace('$ ', '').replace('%', '').replace(',', '').replace('( ', '(').replace(' )', ')')
        line = re.sub(r'(?<![\d.])\b(\d)\s+(\d+)\b', r'\1\2', line) # Fix spaces in numbers that happens pre-2019
        line = line.split()
        line_values = line[len(line) - 9:]
    
        clean_values = ['-' + value[1:-1] if value.startswith('(') and value.endswith(')') else value for value in line_values ]
        clean_values = [np.nan if value == '-' or value == '' else value for value in clean_values]
            
        source = ' '.join(line[0:len(line) - 9])
        clean_values = [source] + clean_values
        data.append(clean_values)

    columns = ['Source', 'Month Actual', 'Month Budget', 'Month Variance', 'Month % Variance', 'FYTD Actual', 'FYTD Budget', 'FYTD Variance', 'FYTD % Variance', 'Total Budgeted FY']
    return pd.DataFrame(data, columns=columns)

In [ ]:
from b_App.b_1_Ingest.ingest_me_general_fund_sources import create_through_time_general_fund_sources
from b_App.b_1_Ingest.ingest_me_general_fund_sources import load_me_general_fund_source_table

In [ ]:
create_through_time_general_fund_sources(2016, 2025)

,2016,2017,2018,2020,2021,2022,2023,2024,2025
Source,,,,,,,,,
Auto Sales Tax Transfer to Highway Fund,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-107534228,-115811095
Cigarette and Tobacco Tax,141464095,144243207,132949700,137331317,146750671,146424162,149909124,144147994,146991431
Corporate Income Tax,137492442,175239114,185737065,216131489,284316774,415817438,451211056,459752873,410118265
Estate Tax,27198153,11717686,13801409,21079344,40399594,34183165,30117577,29051766,85835306
Fines Forfeits & Penalties,20610571,19589175,18402955,9986146,8720806,4905201,10576399,11890588,11407044
Income from Investments,1483723,3592268,6601717,12121418,6748690,9023821,33812410,62564325,68540223
Individual Income Tax,1542687615,1523852981,1595191847,1835972805,2069715243,2580656661,2473478947,2388991095,2723971330
Insurance Companies Tax,81250784,76553592,73469449,82145116,84462691,101673456,114172706,108435700,130061226
Other Revenues,26645824,24747389,4991864,25367559,27863962,47611311,11888085,33515259,19217683


In [ ]:
load_me_general_fund_source_table(2019)

""
